In [ ]:
!jupyter nbconvert --to markdown 7_5_5_Exercises.ipynb

In [ ]:
import sys
import torch.nn as nn
import torch
import warnings
sys.path.append('/home/jovyan/work/d2l_solutions/notebooks/exercises/d2l_utils/')
import d2l
warnings.filterwarnings("ignore")

# 1. Following up on the discussion above, analyze the computational properties of AlexNet.



## 1.1 Compute the memory footprint for convolutions and fully connected layers, respectively. Which one dominates?



## 1.2 Calculate the computational cost for the convolutions and the fully connected layers.



## 1.3 How does the memory (read and write bandwidth, latency, size) affect computation? Is there any difference in its effects for training and inference?



# 2. You are a chip designer and need to trade off computation and memory bandwidth. For example, a faster chip requires more power and possibly a larger chip area. More memory bandwidth requires more pins and control logic, thus also more area. How do you optimize?



# 3. Why do engineers no longer report performance benchmarks on AlexNet?



# 4. Try increasing the number of epochs when training AlexNet. Compared with LeNet, how do the results differ? Why?



# 5. AlexNet may be too complex for the Fashion-MNIST dataset, in particular due to the low resolution of the initial images.



## 5.1 Try simplifying the model to make the training faster, while ensuring that the accuracy does not drop significantly.


## 5.2 Design a better model that works directly on 
 images.



# 6. Modify the batch size, and observe the changes in throughput (images/s), accuracy, and GPU memory.



# 7. Apply dropout and ReLU to LeNet-5. Does it improve? Can you improve things further by preprocessing to take advantage of the invariances inherent in the images?



# 8. Can you make AlexNet overfit? Which feature do you need to remove or change to break training?